In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
import pandas as pd
import chardet

# 불러오기

In [2]:
first = [str(x) for x in Path('./file21').glob('*')]
print(first)
second = [str(x) for x in Path(f'./{first[0]}/').glob('*')]
third = [str(x) for x in Path(f'./{second[0]}/').glob('*')]
# INDIVIDUAL FILES
fourth = [str(x) for x in Path(f'./{third[0]}/').glob('*')]
# XML
fifth = [str(x) for x in Path(f'./{fourth[1]}/').glob('*')]

['file21/KR_OPN_2017']


# 인코딩 탐지

In [3]:
with open(fifth[0], 'rb') as file:
    det = chardet.detect(file.read())
    print(det)



{'encoding': 'EUC-KR', 'confidence': 0.99, 'language': 'Korean'}


# 파싱

In [4]:
with open(fifth[0], 'r', encoding='euc-kr') as file:
    tree = ET.parse(file, parser=ET.XMLParser(encoding='EUC-KR'))
    root = tree.getroot()

In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
# XML 파일 로드하기

def parsing_xml(root):
    data = []

    document_types = root.findall('.//KR_DocumentType')
    for doc_type in document_types:
        document_type = doc_type.text

        # KR_ApplicationInformation 태그 정보 추출
        application_info = root.find('.//KR_ApplicationInformation')
        application_number = application_info.find('.//KR_ApplicationNumber').text
        application_date = application_info.find('.//KR_ApplicationDate').text

        # KR_Inventor 태그 정보 추출 
        inventors = root.findall('.//KR_InventorInformation')
        inventor_names = [inventor.find('.//KR_Name').text for inventor in inventors]
        inventor_address = [inventor.find('.//KR_Address').text for inventor in inventors]
        inventor_postcode = [inventor.find('.//KR_Postcode').text for inventor in inventors]
        inventor_country = [inventor.find('.//KR_Country').text for inventor in inventors]

        # invention-title 태그 정보 추출
        invention_title = root.find('.//invention-title').text

        # technical-field 태그 정보 추출
        technical_field = root.find('.//technical-field').find('.//p').text

        # background-art 태그 정보 추출
        background_art_paragraphs = root.findall('.//background-art//p')
       
        background_art = [paragraph.text for paragraph in background_art_paragraphs] 

        # <tech-problem> 태그 정보 추출
        tech_problem_paragraphs = root.findall('.//tech-problem//p')
        tech_problem = [paragraph.text for paragraph in tech_problem_paragraphs]  

        # <KR_OpenNumber> 
        opennumber = root.find('.//KR_OpenNumber').text

        #KR_OpenDate
        opendate = root.find('.//KR_OpenDate').text

        # KR_Applicant 태그 정보 추출
        applicant = root.find('.//KR_ORGName').text if root.find('.//KR_ORGName') != None else ''
            
        # KR_OpenNumber 및 KR_OpenDate 태그 정보 추출
        publication_number = root.find('.//KR_OpenNumber').text
        publication_date = root.find('.//KR_OpenDate').text

        # KR_IPCInformation 태그의 분류 정보 추출 (복수의 IPC가 있을 수 있으므로 리스트 사용)
        ipc_classifications = root.findall('.//KR_IPCInformation//KR_MainClassification')
        ipc_codes = [ipc.attrib['value'] for ipc in ipc_classifications]

        # tech-solution 태그 정보 추출
        tech_solutions = root.findall('.//tech-solution//p')
        tech_solution = [x.text for x in tech_solutions]

        # advantageous_effects 태그 정보 추출
        advantageous_effects = root.findall('.//advantageous-effects//p')
        advantageous_effect = [x.text for x in advantageous_effects]

        # description-of-drawings태그 정보 추출
        description_of_drawings = root.findall('.//description-of-drawings//p')
        description_of_drawing = [x.text for x in description_of_drawings]

        # description-of-embodiments 태그 정보 추출
        description_of_embodiments = root.findall('.//description-of-embodiments//p')
        description_of_embodiment = [x.text for x in description_of_embodiments]

        # reference-signs-list 태그 정보 추출
        reference_signs_list = root.findall('.//reference-signs-list//p')
        reference_signs = [x.text for x in reference_signs_list]

        # claim-text 태그 정보 추출
        claims_text = root.findall('.//claim//claim-text')
        claims = [x.text for x in claims_text]

        # summary 태그 정보 추출
        summary_text = root.findall('.//summary//p')
        summary = [x.text for x in summary_text]

        # drawing 태그 정보 추출
        drawings_text = root.findall('.//drawings//figure//img')
        drawing = [x.text for x in drawings_text]

        data_row = {
            'KR_Address': inventor_address,
            'KR_Name': inventor_names,
            'KR_invention_title': invention_title,
            'Technical_field' : technical_field,
            'background_art' : background_art,
            'tech_problem' : tech_problem,
            'Technical Solution': tech_solution,
            'Advantageous Effects': advantageous_effect,
            'Description of Drawings': description_of_drawing,
            'Description of Embodiments': description_of_embodiment,
            'Reference Signs List': reference_signs,
            'Claims': claims,
            'Summary': summary,
            'Drawings': drawing,
            'KR_PostCode': inventor_postcode,
            'KR_Country': inventor_country,
            'KR_OpenDate' : opendate,
            'Document Type': document_type,
            'KR_OpenNumber': opennumber,
            'Application Number': application_number,
            'Application Date': application_date,
            'Inventor Name': inventor_names,
            'Applicant': applicant,
            'Publication Number': publication_number,
            'Publication Date': publication_date,
            'IPC Codes': ', '.join(ipc_codes)
        }
        data.append(data_row)

    # 빈 데이터 리스트를 DataFrame으로 변환
    df = pd.DataFrame(data)
    
    return data

In [27]:
datas = []
from tqdm import tqdm
for i in tqdm(second):
    for j in [str(x) for x in Path(f'./{second[0]}/').glob('*')]:
        xml = [str(x) for x in Path(f'./{j}/XML').glob('*.xml')][0]
        with open(xml, 'rb') as file:
            det = chardet.detect(file.read())

        with open(xml, 'r', encoding=det['encoding']) as file:
            tree = ET.parse(file, parser=ET.XMLParser(encoding=det['encoding']))
            root = tree.getroot()
            datas.append(parsing_xml(root))

  0%|          | 0/42 [00:09<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
dd = defaultdict(list)

for d in datas:
    for key, value in d[0].items():
        dd[key].append(value)


In [29]:
pd.DataFrame(dd) 

,KR_Address,KR_Name,KR_invention_title,Technical_field,background_art,tech_problem,Technical Solution,Advantageous Effects,Description of Drawings,Description of Embodiments,...,KR_OpenDate,Document Type,KR_OpenNumber,Application Number,Application Date,Inventor Name,Applicant,Publication Number,Publication Date,IPC Codes
0,"[경기도 수원시 영통구 태장로82번길 32 ,105동305호(망포동,망포마을동수...","[이유영, 강다윤]","근거리 무선 통신을 이용한 카드 결제 시스템, 그 기능을 가지는 이동통신단말기 및 ...",본 발명은 근거리 무선 통신(Near Field Communication: NFC)...,"[최근, 음성 전화 통신뿐만 아니라, 언제 어디서나 인터넷에 접속하여 인터넷상의 정...",[따라서 본 발명의 목적은 NFC 모듈을 구비하고 인터넷에 접속하여 물품을 구매할 ...,[상기와 같은 목적을 달성하기 위한 본 발명의 근거리 무선 통신을 이용한 카드 결제...,[본 발명은 NFC 모듈을 구비하는 이동통신단말기에서 NFC 모듈을 통해 카드의 정...,[도 1은 일반적인 컴퓨터 기반 또는 모바일 기반의 카드 결제 방식을 설명하기 위한...,"[후술하는 본 발명에 대한 상세한 설명은, 본 발명이 실시될 수 있는 특정 실시 예...",...,2017년11월01일,공개특허공보(A),10-2017-0120730,10-2016-0033891,2016년03월22일,"[이유영, 강다윤]",주식회사 페이엠월드,10-2017-0120730,2017년11월01일,I
1,"[서울특별시 서초구 동작대로 112, 101동 1601호(방배동, 한신휴플러스)]",[김민수],플렉서블 표시 장치{FLEXIBLE DISPLAY DEVICE},"본 발명은 표시 장치에 관한 것으로, 특히 플렉서블 표시 장치의 만곡시 이에 가해지...",[플렉서블 표시 장치(또는 접이식 표시 장치)는 휘어지는 가요성 패널을 포함한다. ...,"[본 발명은 상기와 같은 문제점을 해결하기 위하여 안출된 것으로, 접착 필름의 손상...","[상기와 같은 목적을 달성하기 위한 본 발명에 따른 플렉서블 표시 장치는, 표시 패...","[본 발명에 따른 플렉서블 표시 장치는 다음과 같은 효과를 제공한다., 첫째, 접착...",[도 1은 본 발명의 실시예에 따른 플렉서블 표시 장치를 나타낸 도면이다.],"[본 발명의 이점 및 특징, 그리고 그것들을 달성하는 방법은 첨부되는 도면과 함께 ...",...,2017년11월01일,공개특허공보(A),10-2017-0120731,10-2016-0048496,2016년04월21일,[김민수],삼성디스플레이 주식회사,10-2017-0120731,2017년11월01일,I
2,"[대전광역시 유성구 어은로 57 한빛아파트 105동 901호, 서울특별시 강남구 영...","[임탁형, 송락현, 박석주, 이승복, 이종원, 유성빈, 이승호]",원통형 ＳＯＦＣ를 이용한 가압운전 시스템{PRESSURE OPERATING SYST...,"본 발명은 원통형 SOFC를 이용한 가압운전 시스템에 관한 것으로, 더욱 상세하게는...","[본 출원인이 기출원한 특허출원 제10-2015-61463호의 ""튜브셀 기반의 가압...","[본 발명은 상기와 같은 문제점을 개선하기 위하여 발명된 것으로, 비교적 낮은 온도...","[상기와 같은 목적을 달성하기 위하여, 본 발명은 ScSZ(Scandia-Stabi...","[상기와 같은 구성의 본 발명에 따르면, 다음과 같은 효과를 도모할 수 있다., 우...",[도 1은 본 발명의 일 실시예에 따른 원통형 SOFC를 이용한 가압운전 시스템의 ...,"[본 발명의 이점 및 특징, 그리고 그것들을 달성하는 방법은 첨부되는 도면과 함께 ...",...,2017년11월01일,공개특허공보(A),10-2017-0120732,10-2016-0048581,2016년04월21일,"[임탁형, 송락현, 박석주, 이승복, 이종원, 유성빈, 이승호]",한국에너지기술연구원,10-2017-0120732,2017년11월01일,I
3,"[경기도 성남시 분당구 내정로 54 608동 1502호 (정자동,한솔마을주공6단지아...",[김태홍],전동식 파워 스티어링 시스템의 중립 위치 판정 방법{METHOD FOR DETERM...,"본 발명은 전동식 파워 스티어링 시스템의 중립 위치 판정 방법에 관한 것으로, 보다...",[일반적으로 전동식 파워 스티어링(MDPS : Motor Driven Power S...,"[본 발명의 일 측면에 따르면, 본 발명은 상기와 같은 문제점을 해결하기 위해 창작...","[본 발명의 일 측면에 따른 전동식 파워 스티어링 시스템의 중립 위치 판정 방법은,...","[본 발명의 일 측면에 따르면, 본 발명은 차량 주행 시 차속 정보와 토크 정보를 ...",[도 1은 본 발명의 일 실시예에 관련된 전동식 파워 스티어링 시스템의 개략적인 구...,"[이하, 첨부된 도면을 참조하여 본 발명에 따른 전동식 파워 스티어링 시스템의 중립...",...,2017년11월01일,공개특허공보(A),10-2017-0120733,10-2016-0048690,2016년04월21일,[김태홍],현대모비스 주식회사,10-2017-0120733,2017년11월01일,I
4,"[전라북도 익산시 선화1로 31-46, 102동 210호 (마동, 스위트밸리아파트)...","[이은옥, 김쾌만, 강희용, 김다미, 임준영]",정역회전이 가능한 배토기의 기어박스{Forward and reverse rotata...,"본 발명은 정역회전이 가능한 배토기의 기어박스에 관한 것으로서, 구체적으로 간단한 ...",[농기계 중 수요가 가장 많은 기계는 트랙터로 그에 부착되어 사용되어지는 작업기의 ...,[이와 같은 문제점을 해결하기 위한 본 발명은 간단한 조작에 의해 배토날의 회전방향...,[상기와 같은 목적을 달성하기 위한 본 발명은 트랙터의 후방에 설치되어 상기 트랙터...,[본 발명은 클러치부재의 간단한 조작을 통해 배토날의 회전방향을 정회전 또는 역회전...,[도 1은 본 발명의 일실시예인 정역회전이 가능한 배토기의 기어박스를 개략적으로 나...,"[이하, 본 발명의 바람직한 실시예를 첨부한 도면에 의거하여 보다 상세하게 설명하면...",...,2017년11월01일,공개특허공보(A),10-2017-0120734,10-2016-0048717,2016년04월21일,"[이은옥, 김쾌만, 강희용, 김다미, 임준영]",,10-2017-0120734,2017년11월01일,I
5,"[광주광역시 북구 연양로117번길 10, 101동 505호(양산동, 송지트리뷰아파트...","[정광일, 김정인]",리튬 일차전지의 양극 제조방법{METHOD FOR MENUFACTURING A CA...,"본 발명은 리튬 일차전지의 양극 제조방법에 관한 것이며, 보다 상세하게는 2차 열처...","[최근 휴대형 전자기기 등의 소형화, 고성능화 등의 이유로 고에너지 밀도를 가지는 ...",[본 발명은 특정 조성을 포함하는 슬러리를 특정 조건에서 열처리하여 전극을 제조함으...,"[상기 과제를 해결하기 위하여, , 본 발명은 CFx(0.5≤x≤1.5) 불화 탄소...","[본 발명에 따라 제조된 양극을 리튬 일차전지에 적용함으로써, 보다 큰 방전 용량을...",[도 1은 비교예에 따른 전극의 표면을 촬영한 SEM 사진이다.],"[본 발명의 이점 및 특징, 그리고 그것들을 달성하는 방법은 후술하는 실시예들을 참...",...,2017년11월01일,공개특허공보(A),10-2017-0120735,10-2016-0048723,2016년04월21일,"[정광일, 김정인]",주식회사 비츠로셀,10-2017-0120735,2017년11월01일,I
6,"[울산광역시 울주군 언양읍 유니스트길 50, 대전광역시 유성구 대학로 291, KA...","[심성한, 김은주, 박종웅]",무선